In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, LSTM, Dense, Dropout, TimeDistributed, Flatten, Input, concatenate, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define dataset paths
train_data_path = "D:/P_126/data_idumfc/preprocessed_train"
test_data_path = "D:/P_126/data_idumfc/preprocessed_test"

# Define image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Define GRU model
gru_input = Input(shape=(1, img_width, img_height, 3))
gru_features = TimeDistributed(Conv2D(32, (3, 3), activation='relu'))(gru_input)
gru_features = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(gru_features)
gru_features = TimeDistributed(Conv2D(64, (3, 3), activation='relu'))(gru_features)
gru_features = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(gru_features)
gru_features = TimeDistributed(Conv2D(128, (3, 3), activation='relu'))(gru_features)
gru_features = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(gru_features)
gru_features = TimeDistributed(Flatten())(gru_features)
gru_features = GRU(128, activation='relu', return_sequences=False)(gru_features)

# Define Peephole LSTM model
lstm_input = Input(shape=(1, img_width, img_height, 3))
lstm_features = TimeDistributed(Conv2D(32, (3, 3), activation='relu'))(lstm_input)
lstm_features = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(lstm_features)
lstm_features = TimeDistributed(Conv2D(64, (3, 3), activation='relu'))(lstm_features)
lstm_features = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(lstm_features)
lstm_features = TimeDistributed(Conv2D(128, (3, 3), activation='relu'))(lstm_features)
lstm_features = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(lstm_features)
lstm_features = TimeDistributed(Flatten())(lstm_features)
lstm_features = LSTM(128, activation='tanh', recurrent_activation='sigmoid', dropout=0.2, recurrent_dropout=0.2, return_sequences=False)(lstm_features)

# Combine both models in an ensemble
combined_features = concatenate([gru_features, lstm_features])
ensemble_output = Dense(6, activation='softmax')(combined_features)

# Create the final ensemble model
model = tf.keras.models.Model(inputs=[gru_input, lstm_input], outputs=ensemble_output)

# Compile the ensemble model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the ensemble model
model.fit(
    x=[np.expand_dims(train_generator[0][0], axis=1), np.expand_dims(train_generator[0][0], axis=1)],
    y=train_generator[0][1],
    epochs=10,
    steps_per_epoch=312,
    validation_data=([np.expand_dims(test_generator[0][0], axis=1), np.expand_dims(test_generator[0][0], axis=1)], test_generator[0][1]),
    validation_steps=len(test_generator),
)



# Save the trained ensemble model
model.save("ensemble_model.h5")
print("Model saved successfully!")


Found 16050 images belonging to 6 classes.
Found 4110 images belonging to 6 classes.
Epoch 1/10
 32/312 ━━━━━━━━━━━━━━━━━━━━ 9:03 2s/step - accuracy: 0.1485 - loss: 3.3577

C:\ProgramData\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


312/312 ━━━━━━━━━━━━━━━━━━━━ 103s 219ms/step - accuracy: 0.1240 - loss: 2.5436 - val_accuracy: 0.1818 - val_loss: 1.8331
Epoch 2/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 68s 212ms/step - accuracy: 0.1327 - loss: 1.8152 - val_accuracy: 0.0606 - val_loss: 1.8742
Epoch 3/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 75s 236ms/step - accuracy: 0.3271 - loss: 1.6709 - val_accuracy: 0.1212 - val_loss: 2.0023
Epoch 4/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 94s 292ms/step - accuracy: 0.5976 - loss: 0.9309 - val_accuracy: 0.2727 - val_loss: 2.1147
Epoch 5/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 102s 318ms/step - accuracy: 0.9728 - loss: 0.1642 - val_accuracy: 0.3030 - val_loss: 2.6465
Epoch 6/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 138s 302ms/step - accuracy: 0.9728 - loss: 0.0344 - val_accuracy: 0.2424 - val_loss: 3.1401
Epoch 7/10
312/312 ━━━━━━━━━━━━━━━━━━━━ 97s 300ms/step - accuracy: 0.9728 - loss: 0.0104 - val_accuracy: 0.2121 - val_loss: 3.2797
Epoch 8/10
 23/312 ━━━━━━━━━━━━━━━━━━━━ 10:35 2s/step - accuracy: 1.0000 - loss: 0.0057

KeyboardInterrupt: 

In [ ]:
# Evaluate the ensemble model on the test data
y_true = test_generator.classes
y_pred_probs = model.predict([test_generator, test_generator])
y_pred = np.argmax(y_pred_probs, axis=1)

# Calculate test accuracy
test_accuracy = np.sum(y_true == y_pred) / len(y_true)
print(f"Test Accuracy: {test_accuracy}")

# Generate and print classification report
class_names = list(train_generator.class_indices.keys())
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# Generate and print confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, LSTM, Dense, Dropout, TimeDistributed, Flatten, Input, concatenate, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define dataset paths
train_data_path = "D:/P_126/data_idumfc/preprocessed_train"
test_data_path = "D:/P_126/data_idumfc/preprocessed_test"

# Define image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Define GRU model
gru_input = Input(shape=(1, img_width, img_height, 3))
gru_features = TimeDistributed(Conv2D(32, (3, 3), activation='relu'))(gru_input)
gru_features = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(gru_features)
gru_features = TimeDistributed(Conv2D(64, (3, 3), activation='relu'))(gru_features)
gru_features = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(gru_features)
gru_features = TimeDistributed(Conv2D(128, (3, 3), activation='relu'))(gru_features)
gru_features = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(gru_features)
gru_features = TimeDistributed(Flatten())(gru_features)
gru_features = GRU(128, activation='relu', return_sequences=False)(gru_features)

# Define Peephole LSTM model
lstm_input = Input(shape=(1, img_width, img_height, 3))
lstm_features = TimeDistributed(Conv2D(32, (3, 3), activation='relu'))(lstm_input)
lstm_features = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(lstm_features)
lstm_features = TimeDistributed(Conv2D(64, (3, 3), activation='relu'))(lstm_features)
lstm_features = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(lstm_features)
lstm_features = TimeDistributed(Conv2D(128, (3, 3), activation='relu'))(lstm_features)
lstm_features = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(lstm_features)
lstm_features = TimeDistributed(Flatten())(lstm_features)
lstm_features = LSTM(128, activation='tanh', recurrent_activation='sigmoid', dropout=0.2, recurrent_dropout=0.2, return_sequences=False)(lstm_features)

# Combine both models in an ensemble
combined_features = concatenate([gru_features, lstm_features])
ensemble_output = Dense(6, activation='softmax')(combined_features)

# Create the final ensemble model
model = tf.keras.models.Model(inputs=[gru_input, lstm_input], outputs=ensemble_output)

# Compile the ensemble model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the ensemble model
model.fit(
    x=[np.expand_dims(train_generator[0][0], axis=1), np.expand_dims(train_generator[0][0], axis=1)],
    y=train_generator[0][1],
    epochs=20,
    steps_per_epoch=312,
    validation_data=([np.expand_dims(test_generator[0][0], axis=1), np.expand_dims(test_generator[0][0], axis=1)], test_generator[0][1]),
    validation_steps=len(test_generator),
)



# Save the trained ensemble model
model.save("ensemble_model_20.h5")
print("Model saved successfully!")


In [ ]:
# Evaluate the ensemble model on the test data
y_true = test_generator.classes
y_pred_probs = model.predict([test_generator, test_generator])
y_pred = np.argmax(y_pred_probs, axis=1)

# Calculate test accuracy
test_accuracy = np.sum(y_true == y_pred) / len(y_true)
print(f"Test Accuracy: {test_accuracy}")

# Generate and print classification report
class_names = list(train_generator.class_indices.keys())
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# Generate and print confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, LSTM, Dense, Dropout, TimeDistributed, Flatten, Input, concatenate, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define dataset paths
train_data_path = "D:/P_126/data_idumfc/preprocessed_train"
test_data_path = "D:/P_126/data_idumfc/preprocessed_test"

# Define image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Create data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Define GRU model
gru_input = Input(shape=(1, img_width, img_height, 3))
gru_features = TimeDistributed(Conv2D(32, (3, 3), activation='relu'))(gru_input)
gru_features = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(gru_features)
gru_features = TimeDistributed(Conv2D(64, (3, 3), activation='relu'))(gru_features)
gru_features = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(gru_features)
gru_features = TimeDistributed(Conv2D(128, (3, 3), activation='relu'))(gru_features)
gru_features = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(gru_features)
gru_features = TimeDistributed(Flatten())(gru_features)
gru_features = GRU(128, activation='relu', return_sequences=False)(gru_features)

# Define Peephole LSTM model
lstm_input = Input(shape=(1, img_width, img_height, 3))
lstm_features = TimeDistributed(Conv2D(32, (3, 3), activation='relu'))(lstm_input)
lstm_features = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(lstm_features)
lstm_features = TimeDistributed(Conv2D(64, (3, 3), activation='relu'))(lstm_features)
lstm_features = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(lstm_features)
lstm_features = TimeDistributed(Conv2D(128, (3, 3), activation='relu'))(lstm_features)
lstm_features = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(lstm_features)
lstm_features = TimeDistributed(Flatten())(lstm_features)
lstm_features = LSTM(128, activation='tanh', recurrent_activation='sigmoid', dropout=0.2, recurrent_dropout=0.2, return_sequences=False)(lstm_features)

# Combine both models in an ensemble
combined_features = concatenate([gru_features, lstm_features])
ensemble_output = Dense(6, activation='softmax')(combined_features)

# Create the final ensemble model
model = tf.keras.models.Model(inputs=[gru_input, lstm_input], outputs=ensemble_output)

# Compile the ensemble model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the ensemble model
model.fit(
    x=[np.expand_dims(train_generator[0][0], axis=1), np.expand_dims(train_generator[0][0], axis=1)],
    y=train_generator[0][1],
    epochs=30,
    steps_per_epoch=312,
    validation_data=([np.expand_dims(test_generator[0][0], axis=1), np.expand_dims(test_generator[0][0], axis=1)], test_generator[0][1]),
    validation_steps=len(test_generator),
)



# Save the trained ensemble model
model.save("ensemble_model_30.h5")
print("Model saved successfully!")


In [ ]:
# Evaluate the ensemble model on the test data
y_true = test_generator.classes
y_pred_probs = model.predict([test_generator, test_generator])
y_pred = np.argmax(y_pred_probs, axis=1)

# Calculate test accuracy
test_accuracy = np.sum(y_true == y_pred) / len(y_true)
print(f"Test Accuracy: {test_accuracy}")

# Generate and print classification report
class_names = list(train_generator.class_indices.keys())
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# Generate and print confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)